### <숙소관련 변수 전처리 개요>

1. property_type	숙소 유형 (ex: 아파트, 집, 호텔 등) <br>
--> 미활용

2. room_type	방 종류 (Entire home/apt, Private room, Shared room) <br>
--> get_dummies를 활용해 원핫인코딩 <br>
--> 4개 범주 : 'room_type_Entire home/apt', 'room_type_Hotel room', 'room_type_Private room', 'room_type_Shared room'

3.  accommodates	숙소 수용 가능 인원 <br>
--> 유지

4.  bedrooms	침실 개수 <br>
--> 유지

5.  beds	침대 개수 <br>
--> 유지

6.  bathrooms	욕실 수 <br>
--> 침실1개, 침대1개인데 화장실이 10개인 이상치 데이터 1개 삭제

7.  amenities	편의시설 목록 (JSON 형식) <br>
--> 개수 카운팅해서 'amenities_count'파생컬럼 생성

8.  amenities	편의시설 목록 (JSON 형식) <br>
-->인기 어메니티 키워드를 모두 포함하면 1, 아니면 0으로 파생컬럼 생성


### room_type	방 종류 (Entire home/apt, Private room, Shared room) <br>
--> get_dummies를 활용해 원핫인코딩 <br>
--> 4개 범주 : 'room_type_Entire home/apt', 'room_type_Hotel room', 'room_type_Private room', 'room_type_Shared room'

In [2]:
# 기존 room_type 컬럼 유지하면서 더미컬럼을 df에 추가
# prefix='room_type'은 함수에서 생성되는 더미변수 이름 앞에 붙일 접두사 room_type_privae room이렇게 앞에 붙음
# 랜덤포레스트회귀는 다중공선성 문제 없으므로 drop_first=False해서 room_type의 모든 범주가 더미변수로 생성되게 하기

dummies = pd.get_dummies(df['room_type'], prefix='room_type', drop_first=False)
df = pd.concat([df, dummies], axis=1)

NameError: name 'pd' is not defined

### bathrooms	욕실 수 <br>
--> 침실1개, 침대1개인데 화장실이 10개인 이상치 데이터 1개 삭제

In [4]:
#값이 10.5인 이상치 데이터의 인덱스 확인해서 index_to_drop에 담기
index_to_drop = df[df['bathrooms'] == 10.5].index

#인덱스를 기준으로 행 제거하기, 원본에 바로 반영
df.drop(index=index_to_drop, inplace=True)

NameError: name 'df' is not defined

### amenities	편의시설 목록 (JSON 형식) <br>
--> 개수 카운팅해서 'amenities_count'파생컬럼 생성

In [3]:
#ast.literal_eval() 문자열로 된 리시트, 딕셔너리, 숫자, 튜플 등을 실제 파이썬 객체로 안전하게 변환하고 싶을 때 사용
#ast : abstract syntax tree 모듈 -> 파이썬 코드 자체를 문법구조로 분석하는 내장 모듈
#literal_eval : 문자 그대로 평가하는 뜻
#문법을 보고 문자 그대로 평가해서 객체로 바꿔
#apply 각 행에 대해 람다 함수를 적용해
import ast

df['amenities_count'] = df['amenities'].apply(lambda x : len(ast.literal_eval(x))) 
df.head(3)

# 즉 '[]'로 된 문자열 자체를 평가해서 리스트로 변환하고 리스트의 요소를 세는 함수를 각 행에 적용해서 반환

NameError: name 'df' is not defined

### amenities	편의시설 목록 (JSON 형식) <br>
-->인기 어메니티 키워드 6개를 모두 포함하면 1, 아니면 0으로 파생컬럼 생성

In [ ]:
#25년도 7월 기준 에어비앤비 홈페이지 뉴욕시 어메니티 인기 키워드 6개
popular_amenities = [
    'TV',
    'Air conditioning',
    'Wifi',
    'Washer',
    'Free parking',
    'Dryer'
]

#인기 키워드 6종을 모두 포함한 숙소를 1, 아닌 숙소를 0으로 분류
def contains_all_popular_amenity(amenities_str):
    return int(all(keyword in amenities_str for keyword in popular_amenities))

df['has_all_popular_amenity'] = df['amenities'].apply(contains_all_popular_amenity)
